In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import string
import jsonrpclib
from simplejson import loads
server = jsonrpclib.Server("http://localhost:8080")

lem = WordNetLemmatizer()
replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))

In [71]:
result = loads(server.parse('She frowned down at them with dismay and glanced over to where her sister Sansa sat among the other girls. Sansa\'s needlework was exquisite. Everyone said so. "Sansa\'s work is as pretty as she is," Septa Mordane told their lady mother once. "She has such fine, delicate hands." When Lady Catelyn had asked about Arya, the septa had sniffed. "Arya has the hands of a blacksmith."'))
for i in range(len(result['sentences'])):
    print result['sentences'][i]['dependencies']

[[u'root', u'ROOT', u'frowned'], [u'nsubj', u'frowned', u'She'], [u'compound:prt', u'frowned', u'down'], [u'case', u'them', u'at'], [u'nmod:at', u'frowned', u'them'], [u'case', u'dismay', u'with'], [u'nmod:with', u'frowned', u'dismay'], [u'cc', u'frowned', u'and'], [u'conj:and', u'frowned', u'glanced'], [u'case', u'sat', u'over'], [u'mark', u'sat', u'to'], [u'advmod', u'sat', u'where'], [u'nmod:poss', u'Sansa', u'her'], [u'compound', u'Sansa', u'sister'], [u'nsubj', u'sat', u'Sansa'], [u'dep', u'glanced', u'sat'], [u'case', u'girls', u'among'], [u'det', u'girls', u'the'], [u'amod', u'girls', u'other'], [u'nmod:among', u'sat', u'girls']]
[[u'root', u'ROOT', u'exquisite'], [u'nmod:poss', u'needlework', u'Sansa'], [u'case', u'Sansa', u"'s"], [u'nsubj', u'exquisite', u'needlework'], [u'cop', u'exquisite', u'was']]
[[u'root', u'ROOT', u'said'], [u'nsubj', u'said', u'Everyone'], [u'advmod', u'said', u'so']]
[[u'root', u'ROOT', u'told'], [u'nmod:poss', u'work', u'Sansa'], [u'case', u'Sansa', 

Explicit speaker

In [22]:
f = open('./text')
sv = open('./speechverbs')

speechverb = []
for line in sv:
    if line != '\n':
        speechverb.append(line.split('\n')[0].lower())

utterance = []
speaker = []
linenum = []

line_num = 0
for line in f:
    quotation_pos = []
    if '"' in line:
        distance_speechverb = []
        linenum.append(line_num)
        tokens = nltk.word_tokenize(line)
        for i in range(len(tokens)):
            if tokens[i] == '``' or tokens[i] == "''":
                tokens[i] = '"'
                quotation_pos.append(i)
        num_marks = 0
        for j in range(len(tokens)):
            if tokens[j] == '"':
                num_marks = num_marks + 1
            if lem.lemmatize(tokens[j].lower(),'v') in speechverb and num_marks%2 == 0:
                temp = 0
                try:
                    temp = temp + abs(j-quotation_pos[0]) + abs(j-quotation_pos[1])
                except IndexError:
                    temp = temp + abs(j-quotation_pos[0])
                distance_speechverb.append((tokens[j],temp))
        print distance_speechverb
    line_num = line_num +1
                


[('said', 17), ('asked', 25), ('refused', 83), ('insisted', 159)]
[('told', 12), ('whispers', 104)]
[('tell', 18), ('said', 108), ('ask', 218)]
[]
[('said', 29), ('told', 21), ('said', 63)]
[('wondered', 88), ('told', 50), ('said', 12)]
[('asked', 13)]
[('said', 25)]
[('said', 11)]
[('said', 10)]
[('asked', 7)]
[]
[('told', 14)]
[('said', 9)]
[('said', 11)]
[('wondered', 411), ('insisted', 303), ('agreed', 277), ('calling', 231), ('muttered', 37), ('told', 15)]
[('whispered', 14), ('stating', 30)]
[]
[('said', 9)]
[('whispering', 143), ('screams', 79), ('reveal', 17), ('shout', 11), ('sighed', 19)]
[('cries', 6), ('hiss', 134), ('screaming', 156), ('shouted', 170), ('begged', 176), ('voice', 244), ('calling', 272), ('snarling', 304), ('growling', 308), ('snap', 316), ('shrieks', 338)]
[('said', 9)]
[('said', 7)]
[('announced', 8)]
[('said', 8)]
[('told', 15)]
[]
[('said', 8)]
[('urged', 12)]
[('answered', 9)]
[('said', 11)]
[('said', 14)]
[]
[('agreed', 11)]
[('asked', 13)]
[]
[('expla